In [14]:
# Here are some libraries that we'll need for this process
import pandas as pd
from datetime import datetime
# import pyodbc as dbapi
from tqdm import tqdm
from IPython.core.display import Image
from IPython import display
import requests
import os.path
from datetime import timedelta
import json
from collections import defaultdict
import re
import numpy as np
import matplotlib.pyplot as plt
import csv
import requests
import pprint
pp = pprint.PrettyPrinter(indent=4)
import snipeit
# from snipeit import Models

server='https://sawpit.app'
token='yourtokenhere'

In [ ]:
#load first sheet of excel into a dataframe
df = pd.read_excel('Master Inventory - RMJC 4-15-2023.xlsx', index_col=0, sheet_name=0) 
#extract the unique values of Summary Name which will become the unique models 

In [ ]:
#show first row of xl spreadsheet
for index,row in df.iterrows():
    print(index,row)
    break

In [ ]:
def summaryToModel(s):
    
    # we split this string into   "POC - FOHC - 6x6x10"
    # print(s)
    #remove whitespace
    s = s.split('-')
    species = s[0].strip()
    boxedHeart = s[1].strip()
    dims = ''
    if len(s) > 2:
        dims = s[2].strip()
    summary = ' - '.join([species,boxedHeart,dims])
    # category 2 is heavy timber
    manufacturer_id = 1 # 1 is Unknown
    if species ==  "DF":
        manufacturer_id = 2
    if species ==  "POC":
        manufacturer_id = 3
    if species ==  "AYC":
        manufacturer_id = 4
    if species ==  "WRC":
        manufacturer_id = 5
    if species ==  "WO":
        manufacturer_id = 6
    d = {"name":summary, "category_id":2, "manufacturer_id":manufacturer_id, "fieldset_id":2}
    return d


#get a list of unique models
uniqueModels = list(map(summaryToModel, df["Summary Name"].unique().tolist()))
print(len(uniqueModels))
pp.pprint(uniqueModels[0])

In [ ]:
#upload the unique models to sawpit
uploadModels = False
if uploadModels:
    M = snipeit.Models()
    sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
    for m in uniqueModels:
        summaryMatch = sumRegex.match(m['name'])
        if(summaryMatch is None):
            #invalid name, skip it
            continue
        r = M.create(server, token,  json.dumps(m))
        if("error" in r):
            print(r)


In [ ]:
#load a dict that maps name to model ids
M = snipeit.Models()
modelsDict = json.loads( M.get(server, token, 5000) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
model_ids = defaultdict() #default of None
for row in modelsDict['rows']:
    model_ids[row['name']]=row['id']

model_ids

In [ ]:
#load a dict that maps company name to company id 
C = snipeit.Company()
companiesDict = json.loads( C.get(server, token) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
company_ids = defaultdict() #default of None
for row in companiesDict['rows']:
    company_ids[row['name']]=row['id']

company_ids

In [ ]:
#load a dict that maps supplier name to supplier id 
# S = snipeit.Supplier()
# suppliersDict = json.loads( S.get(server, token) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
supplier_ids = defaultdict(lambda: 8,{
    'Delson':1,
    'Bluelinx':2,
    'Rosboro':3,
    'Boulder Lumber':4,
    'Hull-Oakes':5,
    'Customer':6,
    'TETW':7,
    'Unknown':8,
}) 

supplier_ids['Delson']

In [ ]:
#define functions needed for upload
def strip(s):
    if isinstance(s, str):
        s = s.strip()
        if '-' == s: # change dash into blank
            return None
        return s
    return s

def formatDate(s):
    if isinstance(s, str):
        m, d, y = s.strip().split('/')
        return y+'-'+m.rjust(2,'0')+'-'+d.rjust(2,'0')
    return s

def getDims(row):
    summary = row['Summary Name']
    s = summary.split('-')
    species = s[0].strip()
    boxedHeart = s[1].strip()
    dims = None
    if len(s) > 2:
        dims = s[2].strip()
    return dims

def csvToAsset(row):
    summary = row['Summary Name']
    status_id = 10 #10 is imported, but does not have new label yet
    if 'Used' in row['Status']:
        status_id = 9
    # print("supplier ",strip(row['Supplier']),supplier_ids[strip(row['Supplier'])])
    d = {"asset_tag":row['SRL_OLD'], 
        "name":getDims(row), #We set the name to just the dims to help with sorting 
        "serial":row['SRL_OLD'], 
        "status_id":status_id, 
        "model_id": model_ids[summary], 
        "company_id": company_ids[strip(row['Inventory'])], 
        "supplier_id": supplier_ids[strip(row['Supplier'])], 
        "location_id": 1, # 1 is unknown
        "archived": False,
        "warranty_months": None,
        "depreciate": False,
        "requestable": True,
        "rtd_location_id": None,
        "_snipeit_bdf_8": row['BDF'],
        "_snipeit_bdf_cost_10": row['BDF Cost'],
        "_snipeit_condition_9": row['Condition'],
        "_snipeit_date_used_14": row['Date Used'],
        "_snipeit_dryness_3": row['Dryness'],
        "_snipeit_dryness_today_13": row['Dryness Today'],
        "_snipeit_freight_11": row['Freight Cost'],
        "_snipeit_grade_2": row['Grade'],
        "_snipeit_height_5": row['Height'],
        "_snipeit_length_7": row['Length'],
        "_snipeit_markup_12": row['Markup'],
        "_snipeit_project_15": row['Project'],
        "_snipeit_width_4": row['Width'],
        "order_number": row['Order'],
        "book_value": row['Sell'],
        "purchase_cost": row['Value'],
        "purchase_date": formatDate(row['Date']),
    }
    #strip whitespace from values
    cleanDict =  {k: strip(v) for k, v in d.items()}
    return cleanDict

def getID(tag):
    try:
        A = snipeit.Assets()
        return A.getID(server, token, tag)
    except:
        return None


def uploadAssetIfNotExist(asset,update=False):
    tag = asset['asset_tag']
    A = snipeit.Assets()
    existing_id = getID(tag)
    # pp.pprint(asset)
    try:
        if existing_id is None:
            r = A.create(server, token, json.dumps(asset))
            if 'error' in r:
                print(f'uploading {tag} status error')
                print(r)
                r = json.loads(r)
                print(f'{tag} error msg = ' + r["messages"])
            else:
                print(f'{tag} uploaded ok')
              
            # 
            # r = A.create(server, token, json.dumps(asset))
            # print(r)
        else:
            if(update):
                r = A.updateDevice(server, token, str(existing_id) , json.dumps(asset))
                if r:
                    print(f'{tag} updated ok')
            else:
                print(f'{tag} exists, skipping upload')
    except:
        print(f'Exception uploading {tag} ')


        
def getJson(tag):
    existing_id = getID(tag)
    A = snipeit.Assets()
    results = A.getDetailsByID(server, token, existing_id)
    jsonData = json.loads((results).decode('utf-8').replace("'",'"'))
    pp.pprint(jsonData)


getJson('RMT 221001 0067')

In [17]:
#this cell will upload fake maps and gps data

#root@limbertimber:/var/www/html/snipeit/public/uploads/assets# ls -la asset-image-1092-OYQSyNFi3U.jpg
#-rw-r--r-- 1 www-data www-data 94952 Aug 19 15:12 asset-image-1092-OYQSyNFi3U.jpg


import random
import requests

def getRandMapImg():
    mapGrids="B4,C1,C6,C7,C8,CA,CG,CH,D2,D3,D4,D5,E2,E3,E4,E5,E6,E9,EC,G5,G8,GA,GB,H1,H6,H7,H8,H9,HA,HB,HC,HD,HE,HF,HG,HL,HI,HH,HJ".split(',')
    grid = random.choice(mapGrids)
    fn = f"https://sawpit.app/uploads/maps/{grid}.jpg"
    #pscp -r G:\work\snipe-it\rmjImport\maps root@sawpit.app:/var/www/html/snipeit/public/uploads
    #chmod -R 644 /var/www/html/snipeit/public/uploads/maps
    #chown -R www-data:www-data /var/www/html/snipeit/public/uploads/maps
    return fn

def setRandomMapImg(tag):
    #TODO: set better gps cords
    existing_id = getID(tag)
    asset = {
        'image':getRandMapImg(),
        '_snipeit_lastgpsping_16':'@39.9898241,-105.0751819'
        }
    A = snipeit.Assets()
    print(asset)
    results = updateDevice(str(existing_id) , json.dumps(asset))
    print(results)

def updateDevice(DeviceID, payload):
    url = server + '/api/v1/hardware/' + DeviceID
    headers = {
        "accept": "application/json",
        "Authorization": 'Bearer ' + token,
        "content-type": "application/json"
    }

    results = requests.patch(url, headers=headers, data=payload)
    return results.content

InventoryCSVfn = 'Master Inventory - RMJC 4-15-2023 - Inventory Master List.csv'

skiprows = 0
with open(InventoryCSVfn, newline='') as csvfile:
    sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
    reader = csv.DictReader(csvfile)
    rowcount = 0
    for row in reader:
        rowcount+=1
        if rowcount < skiprows:
            continue

        if(len(row['SRL_OLD']) < 3):
            print(f"Row {rowcount} is missing SRL_OLD, skipping it")
            continue
        summaryMatch = sumRegex.match(row['Summary Name'])
        if(summaryMatch is None):
            print(f"Row {rowcount} has bad summary {row['Summary Name']}, skipping it")
            continue
        setRandomMapImg(row['SRL_OLD'])
        if rowcount > 10:
            break


{'image': 'https://sawpit.app/uploads/maps/H6.jpg', '_snipeit_lastgpsping_16': '@39.9898241,-105.0751819'}
b'{"message":"The given data was invalid.","errors":{"image":["The image must be a file of type: png, gif, jpg, jpeg, svg, bmp, svg+xml, webp."]}}'
{'image': 'https://sawpit.app/uploads/maps/E3.jpg', '_snipeit_lastgpsping_16': '@39.9898241,-105.0751819'}
b'{"message":"The given data was invalid.","errors":{"image":["The image must be a file of type: png, gif, jpg, jpeg, svg, bmp, svg+xml, webp."]}}'
{'image': 'https://sawpit.app/uploads/maps/GA.jpg', '_snipeit_lastgpsping_16': '@39.9898241,-105.0751819'}
b'{"message":"The given data was invalid.","errors":{"image":["The image must be a file of type: png, gif, jpg, jpeg, svg, bmp, svg+xml, webp."]}}'
{'image': 'https://sawpit.app/uploads/maps/H6.jpg', '_snipeit_lastgpsping_16': '@39.9898241,-105.0751819'}
b'{"message":"The given data was invalid.","errors":{"image":["The image must be a file of type: png, gif, jpg, jpeg, svg, bmp, 

In [ ]:
#this cell will do the upload

InventoryCSVfn = 'Master Inventory - RMJC 4-15-2023 - Inventory Master List.csv'

#TODO: strip whitespace from column names
skiprows = 0
with open(InventoryCSVfn, newline='') as csvfile:
    sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
    reader = csv.DictReader(csvfile)
    rowcount = 0
    for row in reader:
        rowcount+=1
        if rowcount < skiprows:
            continue

        if(len(row['SRL_OLD']) < 3):
            print(f"Row {rowcount} is missing SRL_OLD, skipping it")
            continue
        summaryMatch = sumRegex.match(row['Summary Name'])
        if(summaryMatch is None):
            print(f"Row {rowcount} has bad summary {row['Summary Name']}, skipping it")
            continue
        asset = csvToAsset(row)
        uploadAssetIfNotExist(asset)
        if rowcount > 1000:
            break

    


In [ ]:
#print first 5 assests
A = snipeit.Assets()
a = json.loads(A.get(server, token, 5)) # With a limit of results
pp.pprint(a['rows'][3])
